In [ ]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk
import threading
import logging
from collections import deque

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class ImageProcessor:
    def __init__(self):
        self.face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.prev_gray = None
        self.prev_transform = None
        self.transform_differences = deque(maxlen=10)
        self.target_size = (640, 480)
        self.prev_frames = deque(maxlen=5)
        self.key_points = None
        self.its_values = []  # Add list to store ITS values
    
    def resize_frame(self, frame: np.ndarray) -> np.ndarray:
        if frame is None or frame.size == 0:
            logger.warning("Empty frame received in resize_frame")
            return frame
            
        try:
            return cv2.resize(frame, self.target_size)
        except cv2.error as e:
            logger.error(f"Error resizing frame: {e}")
            return frame
            
    def adjust_image(self, frame: np.ndarray, gamma: float, brightness: float, contrast: float) -> np.ndarray:
        if frame is None or frame.size == 0:
            logger.warning("Empty frame received in adjust_image")
            return frame
            
        try:
            inv_gamma = 1.0 / gamma
            table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
            frame = cv2.LUT(frame, table)
            return cv2.convertScaleAbs(frame, alpha=contrast, beta=brightness)
        except Exception as e:
            logger.error(f"Error adjusting image: {e}")
            return frame
            
    def detect_and_process_faces(self, frame: np.ndarray, detect: bool, blur: bool) -> np.ndarray:
        if frame is None or frame.size == 0:
            logger.warning("Empty frame received in detect_and_process_faces")
            return frame

        try:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = self.face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            for (x, y, w, h) in faces:
                # Add boundary checks
                x = max(0, x)
                y = max(0, y)
                w = min(w, frame.shape[1] - x)
                h = min(h, frame.shape[0] - y)

                if blur and w > 0 and h > 0:  # Only blur if region is valid
                    try:
                        face_region = frame[y:y + h, x:x + w]
                        if face_region.size > 0:  # Check if region is not empty
                            blurred_region = cv2.GaussianBlur(face_region, (99, 99), 30)
                            frame[y:y + h, x:x + w] = blurred_region
                    except cv2.error as e:
                        logger.error(f"Error applying blur to face region: {e}")
                        continue

                if detect:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            return frame

        except Exception as e:
            logger.error(f"Error in detect_and_process_faces: {e}")
            return frame
            
    def optical_flow_stabilization(self, frame: np.ndarray) -> tuple[np.ndarray, float]:
        frame = self.resize_frame(frame)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if self.prev_gray is None:
            self.prev_gray = gray
            return frame, 0.0

        try:
            flow = cv2.calcOpticalFlowFarneback(self.prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            dx, dy = np.mean(flow[..., 0]), np.mean(flow[..., 1])
            transform = np.array([[1, 0, -dx], [0, 1, -dy], [0, 0, 1]])
            height, width = frame.shape[:2]
            stabilized_frame = cv2.warpAffine(frame, transform[:2], (width, height))
            its_value = self.update_its(transform)
            self.prev_gray = gray
            self.prev_transform = transform
            return stabilized_frame, its_value
        except cv2.error as e:
            logger.error(f"Error in optical flow calculation: {e}")
            self.prev_gray = gray
            return frame, 0.0

    def moving_average_stabilization(self, frame: np.ndarray) -> tuple[np.ndarray, float]:
        self.prev_frames.append(frame)
        if len(self.prev_frames) < 2:
            return frame, 0.0
        
        stabilized = np.mean(self.prev_frames, axis=0).astype(np.uint8)
        its_value = np.mean([np.sum(np.abs(stabilized - f)) for f in self.prev_frames]) / (frame.shape[0] * frame.shape[1])
        return stabilized, its_value

    def feature_based_stabilization(self, frame: np.ndarray) -> tuple[np.ndarray, float]:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Initialize ORB detector
        orb = cv2.ORB_create()
        
        if self.key_points is None:
            self.key_points = gray
            return frame, 0.0
            
        try:
            # Find keypoints and descriptors
            kp1, des1 = orb.detectAndCompute(self.key_points, None)
            kp2, des2 = orb.detectAndCompute(gray, None)
            
            if des1 is None or des2 is None or len(kp1) < 2 or len(kp2) < 2:
                self.key_points = gray
                return frame, 0.0
            
            # Create matcher and match features
            bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
            matches = bf.match(des1, des2)
            
            # Sort matches by distance
            matches = sorted(matches, key=lambda x: x.distance)
            
            # Get matched keypoints
            src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
            
            # Calculate transformation matrix
            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
            
            if M is not None:
                # Apply transformation
                h, w = frame.shape[:2]
                stabilized_frame = cv2.warpPerspective(frame, M, (w, h))
                its_value = np.sum(np.abs(M - np.eye(3))) / 9  # Measure of transformation magnitude
                self.key_points = gray
                return stabilized_frame, its_value
            
        except Exception as e:
            logger.error(f"Error in feature-based stabilization: {e}")
        
        self.key_points = gray
        return frame, 0.0

    def update_its(self, transform: np.ndarray) -> float:
        if self.prev_transform is not None:
            diff = np.linalg.norm(transform - self.prev_transform)
            self.transform_differences.append(diff)
            return np.mean(self.transform_differences)
        return 0.0

    def reset_stabilization(self):
        self.prev_gray = None
        self.prev_transform = None
        self.transform_differences.clear()
        self.prev_frames.clear()
        self.key_points = None
        self.its_values.clear()  # Clear ITS values on reset
    def get_average_its(self):
        return np.mean(self.its_values) if self.its_values else 0.0


class VideoProcessingDialog:
    def __init__(self, parent):
        self.dialog = tk.Toplevel(parent)
        self.dialog.title("Video Processing Options")
        self.dialog.geometry("400x500")
        self.dialog.transient(parent)
        self.dialog.grab_set()  # Make dialog modal
        
        # Processing options
        self.face_detection = tk.BooleanVar(value=False)
        self.face_blur = tk.BooleanVar(value=False)
        self.auto_adjust = tk.BooleanVar(value=False)
        self.stabilization = tk.StringVar(value="Off")
        self.save_output = tk.BooleanVar(value=False)
        self.output_path = None
        
        self.setup_ui()
        self.result = None

    def setup_ui(self):
        # Main frame
        main_frame = ttk.Frame(self.dialog, padding="20")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Title
        title_label = ttk.Label(main_frame, text="Select Processing Features", 
                               font=('Helvetica', 12, 'bold'))
        title_label.pack(pady=(0, 20))
        
        # Processing options frame
        options_frame = ttk.LabelFrame(main_frame, text="Processing Options", padding="10")
        options_frame.pack(fill=tk.X, pady=(0, 20))
        
        ttk.Checkbutton(options_frame, text="Face Detection", 
                       variable=self.face_detection).pack(anchor="w", pady=5)
        ttk.Checkbutton(options_frame, text="Face Blur", 
                       variable=self.face_blur).pack(anchor="w", pady=5)
        ttk.Checkbutton(options_frame, text="Auto Adjust Lighting", 
                       variable=self.auto_adjust).pack(anchor="w", pady=5)
        
        # Stabilization options
        ttk.Label(options_frame, text="Stabilization Method:").pack(anchor="w", pady=(10, 5))
        methods = ["Off", "Optical Flow", "Moving Average", "Feature Based"]
        for method in methods:
            ttk.Radiobutton(options_frame, text=method, value=method, 
                           variable=self.stabilization).pack(anchor="w", padx=20, pady=2)
        
        # Save options
        save_frame = ttk.LabelFrame(main_frame, text="Save Options", padding="10")
        save_frame.pack(fill=tk.X, pady=(0, 20))
        
        ttk.Checkbutton(save_frame, text="Save Processed Video", 
                       variable=self.save_output, 
                       command=self.toggle_save_options).pack(anchor="w", pady=5)
        
        self.save_button = ttk.Button(save_frame, text="Select Save Location", 
                                    command=self.select_save_location, state='disabled')
        self.save_button.pack(anchor="w", pady=5)
        
        self.save_path_label = ttk.Label(save_frame, text="No save location selected", 
                                       wraplength=350)
        self.save_path_label.pack(anchor="w", pady=5)
        
        # Buttons frame
        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.pack(fill=tk.X, pady=(20, 0))
        
        ttk.Button(buttons_frame, text="Cancel", 
                  command=self.cancel).pack(side=tk.RIGHT, padx=5)
        ttk.Button(buttons_frame, text="Process", 
                  command=self.process).pack(side=tk.RIGHT, padx=5)

    def toggle_save_options(self):
        if self.save_output.get():
            self.save_button.configure(state='normal')
        else:
            self.save_button.configure(state='disabled')
            self.output_path = None
            self.save_path_label.configure(text="No save location selected")

    def select_save_location(self):
        path = filedialog.asksaveasfilename(
            defaultextension=".mp4",
            filetypes=[("MP4 files", "*.mp4"), ("All files", "*.*")]
        )
        if path:
            self.output_path = path
            self.save_path_label.configure(text=f"Save location: {path}")

    def cancel(self):
        self.result = None
        self.dialog.destroy()

    def process(self):
        if self.save_output.get() and not self.output_path:
            messagebox.showerror("Error", "Please select a save location first!")
            return
            
        self.result = {
            'face_detection': self.face_detection.get(),
            'face_blur': self.face_blur.get(),
            'auto_adjust': self.auto_adjust.get(),
            'stabilization': self.stabilization.get(),
            'save_output': self.save_output.get(),
            'output_path': self.output_path
        }
        self.dialog.destroy()


class CameraApp:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("Video Processing App")
        self.window.geometry("1200x700")

        # Initialize video-related variables first
        self.video_cap = None
        self.stabilization_method = "Off"
        self.stop_video_thread = threading.Event()
        self.video_thread = None
        self.lock = threading.Lock()
        self.is_video_loaded = False
        self.processing_video = False

        # Then initialize the rest
        self.image_processor = ImageProcessor()
        self.setup_variables()
        self.setup_ui()
        self.setup_camera()
        
    def update_display(self, frame, its_value):
        """Update the UI with the current frame"""
        try:
            if frame is None or frame.size == 0:
                logger.warning("Empty frame received in update_display")
                return

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image)
            image = ImageTk.PhotoImage(image)
            self.video_label.config(image=image)
            self.video_label.image = image
            self.its_label.config(text=f"ITS: {its_value:.2f}")
        except Exception as e:
            logger.error(f"Error updating display: {e}")
            
    def run(self):
        """Start the application main loop"""
        try:
            # Set up the window close handler
            self.window.protocol("WM_DELETE_WINDOW", self.close)
            # Start the main event loop
            self.window.mainloop()
        except Exception as e:
            logger.error(f"Error in main loop: {e}")
        finally:
            # Ensure cleanup happens
            self.close()
    def setup_variables(self):
        self.face_detection_on = tk.BooleanVar(value=True)
        self.blur_on = tk.BooleanVar(value=False)
        self.auto_adjust_on = tk.BooleanVar(value=True)
        self.gamma_value = tk.DoubleVar(value=1.0)
        self.brightness = tk.DoubleVar(value=0)
        self.contrast = tk.DoubleVar(value=1.0)

    def setup_ui(self):
        main_frame = ttk.Frame(self.window, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        self.video_label = ttk.Label(main_frame)
        self.video_label.pack(pady=5)
        self.its_label = ttk.Label(main_frame, text="ITS: 0.0")
        self.its_label.pack(pady=2)
        controls_frame = ttk.LabelFrame(main_frame, text="Controls", padding="5")
        controls_frame.pack(fill=tk.X, pady=5)
        ttk.Checkbutton(controls_frame, text="Face Detection", variable=self.face_detection_on).pack(side=tk.LEFT, padx=5)
        ttk.Checkbutton(controls_frame, text="Face Blur", variable=self.blur_on).pack(side=tk.LEFT, padx=5)
        ttk.Checkbutton(controls_frame, text="Auto Adjust", variable=self.auto_adjust_on).pack(side=tk.LEFT, padx=5)

        self.stabilization_var = tk.StringVar(value="Off")
        stabilization_menu = ttk.OptionMenu(controls_frame, self.stabilization_var, "Off", 
                                          "Off", "Optical Flow", "Moving Average", "Feature Based",
                                          command=self.change_stabilization_method)
        stabilization_menu.pack(side=tk.LEFT, padx=5)

        self.load_video_btn = ttk.Button(controls_frame, text="Load Video", command=self.load_video)
        self.load_video_btn.pack(side=tk.LEFT, padx=5)

        sliders_frame = ttk.LabelFrame(main_frame, text="Image Adjustments", padding="5")
        sliders_frame.pack(fill=tk.X, pady=5)
        self.create_slider(sliders_frame, "Gamma", self.gamma_value, 0.5, 2.0, 0.1)
        self.create_slider(sliders_frame, "Brightness", self.brightness, -100, 100, 1)
        self.create_slider(sliders_frame, "Contrast", self.contrast, 0.5, 2.0, 0.1)

    def create_slider(self, parent, label, variable, min_val, max_val, resolution):
        frame = ttk.Frame(parent)
        frame.pack(fill=tk.X, pady=2)
        ttk.Label(frame, text=label).pack(side=tk.LEFT, padx=5)
        ttk.Scale(frame, from_=min_val, to=max_val, variable=variable, orient=tk.HORIZONTAL).pack(side=tk.LEFT, fill=tk.X, expand=True, padx=5)

    def change_stabilization_method(self, method):
        self.stabilization_method = method
        self.image_processor.reset_stabilization()

    def setup_camera(self):
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            logger.error("Failed to open webcam")
            messagebox.showerror("Error", "Could not open webcam")
            return
        self.start_video_thread()


    def load_video(self):
        video_path = filedialog.askopenfilename(
            title="Select Video File",
            filetypes=[("MP4 files", "*.mp4"), ("All files", "*.*")]
        )
        
        if video_path:
            # Show processing options dialog
            dialog = VideoProcessingDialog(self.window)
            self.window.wait_window(dialog.dialog)
            
            if dialog.result is None:  # User cancelled
                return
                
            # Apply selected settings
            self.face_detection_on.set(dialog.result['face_detection'])
            self.blur_on.set(dialog.result['face_blur'])
            self.auto_adjust_on.set(dialog.result['auto_adjust'])
            self.stabilization_var.set(dialog.result['stabilization'])
            
            # Open video
            if self.video_cap is not None:
                self.video_cap.release()
            
            self.video_cap = cv2.VideoCapture(video_path)
            if not self.video_cap.isOpened():
                logger.error("Failed to open video")
                messagebox.showerror("Error", "Failed to open video file")
                self.video_cap = None
                return
                
            self.image_processor.reset_stabilization()
            self.is_video_loaded = True
            
            # If save is enabled, set up video writer
            if dialog.result['save_output']:
                self.setup_video_writer(dialog.result['output_path'])
                threading.Thread(target=self.process_and_save_video, 
                               args=(video_path, dialog.result)).start()

    def setup_video_writer(self, output_path):
        fps = self.video_cap.get(cv2.CAP_PROP_FPS)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        self.video_writer = cv2.VideoWriter(
            output_path, 
            fourcc, 
            fps, 
            self.image_processor.target_size
        )

    def process_and_save_video(self, video_path, settings):
        try:
            self.processing_video = True
            cap = cv2.VideoCapture(video_path)
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            processed_frames = 0
            
            self.stabilization_method = settings['stabilization']
            
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                    
                processed_frame, its_value = self.process_frame(frame)
                
                # Add ITS value display to the frame
                h, w = processed_frame.shape[:2]
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.6
                thickness = 2
                
                # Display instantaneous ITS (bottom left)
                inst_its_text = f"ITS: {its_value:.2f}"
                cv2.putText(processed_frame, inst_its_text, (10, h-10), 
                           font, font_scale, (255, 255, 255), thickness)
                
                # Calculate and display average ITS (bottom right)
                avg_its = self.image_processor.get_average_its()
                avg_its_text = f"Avg ITS: {avg_its:.2f}"
                text_size = cv2.getTextSize(avg_its_text, font, font_scale, thickness)[0]
                cv2.putText(processed_frame, avg_its_text, 
                           (w - text_size[0] - 10, h-10), 
                           font, font_scale, (255, 255, 255), thickness)
                
                self.video_writer.write(processed_frame)
                
                processed_frames += 1
                progress = (processed_frames / total_frames) * 100
                
                if processed_frames % 30 == 0:
                    logger.info(f"Processing progress: {progress:.1f}%")
            
            cap.release()
            self.video_writer.release()
            
            # Reset everything
            self.is_video_loaded = False
            self.video_cap = None
            self.stabilization_method = "Off"
            self.stabilization_var.set("Off")
            self.image_processor.reset_stabilization()
            
            messagebox.showinfo("Success", "Video processing completed! Saved processed video.")
            
        except Exception as e:
            logger.error(f"Error processing video: {e}")
            messagebox.showerror("Error", f"Failed to process video: {str(e)}")
        finally:
            if hasattr(self, 'video_writer'):
                self.video_writer.release()
            self.processing_video = False

    
    def process_frame(self, frame: np.ndarray) -> tuple[np.ndarray, float]:
        if frame is None or frame.size == 0:
            logger.warning("Empty frame received in process_frame")
            return frame, 0.0

        try:
            frame = self.image_processor.resize_frame(frame)
            if frame is None or frame.size == 0:
                return frame, 0.0

            if self.auto_adjust_on.get():
                try:
                    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    mean_brightness = np.mean(gray)
                    if mean_brightness < 80:
                        self.gamma_value.set(1.5)
                        self.brightness.set(50)
                        self.contrast.set(1.5)
                    elif mean_brightness > 180:
                        self.gamma_value.set(0.7)
                        self.brightness.set(0)
                        self.contrast.set(1.0)
                    else:
                        self.gamma_value.set(1.0)
                        self.brightness.set(0)
                        self.contrast.set(1.0)
                except Exception as e:
                    logger.error(f"Error in auto adjustment: {e}")

            frame = self.image_processor.adjust_image(frame, self.gamma_value.get(), 
                                                    self.brightness.get(), self.contrast.get())
            if frame is None or frame.size == 0:
                return frame, 0.0

            its_value = 0.0
            if self.stabilization_method == "Optical Flow":
                frame, its_value = self.image_processor.optical_flow_stabilization(frame)
            elif self.stabilization_method == "Moving Average":
                frame, its_value = self.image_processor.moving_average_stabilization(frame)
            elif self.stabilization_method == "Feature Based":
                frame, its_value = self.image_processor.feature_based_stabilization(frame)

            # Store ITS value for averaging
            if its_value > 0:
                self.image_processor.its_values.append(its_value)

            if frame is None or frame.size == 0:
                return frame, 0.0

            if self.face_detection_on.get() or self.blur_on.get():
                frame = self.image_processor.detect_and_process_faces(frame, 
                                                                   self.face_detection_on.get(), 
                                                                   self.blur_on.get())

            return frame, its_value

        except Exception as e:
            logger.error(f"Error in process_frame: {e}")
            return frame, 0.0
    
    def update_video(self):
        while not self.stop_video_thread.is_set():
            with self.lock:
                if self.processing_video:
                    continue
                    
                if self.is_video_loaded and self.video_cap and self.video_cap.isOpened():
                    ret, frame = self.video_cap.read()
                    if not ret:
                        self.video_cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                        continue
                else:
                    ret, frame = self.cap.read()
                    if not ret:
                        continue
                
                try:
                    frame, its_value = self.process_frame(frame)
                    self.its_label.config(text=f"ITS: {its_value:.2f}")
                    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    image = Image.fromarray(image)
                    image = ImageTk.PhotoImage(image)
                    self.video_label.config(image=image)
                    self.video_label.image = image
                except Exception as e:
                    logger.error(f"Error processing frame: {e}")

    
    def start_video_thread(self):
        if self.video_thread and self.video_thread.is_alive():
            self.stop_video_thread.set()
            self.video_thread.join()
        self.stop_video_thread.clear()
        self.video_thread = threading.Thread(target=self.update_video)
        self.video_thread.daemon = True  # This ensures the thread stops when the main program exits
        self.video_thread.start()

    def close(self):
        logger.info("Closing application...")
        self.stop_video_thread.set()
        if self.video_thread and self.video_thread.is_alive():
            self.video_thread.join()
        if hasattr(self, 'cap') and self.cap.isOpened():
            self.cap.release()
        if self.video_cap and self.video_cap.isOpened():
            self.video_cap.release()
        if hasattr(self, 'video_writer'):
            self.video_writer.release()
        cv2.destroyAllWindows()
        self.window.destroy()
        logger.info("Application closed successfully")


if __name__ == "__main__":
    try:
        logging.info("Starting Video Processing App")
        app = CameraApp()
        app.run()
    except Exception as e:
        logger.error(f"Application failed to start: {e}")
        messagebox.showerror("Error", f"Application failed to start: {e}")
    finally:
        cv2.destroyAllWindows()

2024-11-09 12:19:53,619 - INFO - Starting Video Processing App
